<a href="https://colab.research.google.com/github/lakshmangn/cat-dog-classifier/blob/main/cat_dog_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q kaggle
! sudo pip install h5py
from google.colab import files

files.upload()

In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

! kaggle competitions download -c 'dogs-vs-cats'

In [ ]:
! mkdir Datasets
! unzip train.zip -d Datasets
! unzip test1.zip -d Datasets

In [ ]:
import os
import pandas as pd 
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.preprocessing import image as image_process

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l2
from keras.preprocessing.image import load_img

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
tf.__version__

In [ ]:
# Training dataframe
filenames = os.listdir("Datasets/train/")
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        categories.append(1)
    else:
        categories.append(0)
df = pd.DataFrame({
    'filename': filenames,
    'category_binary': categories
})
df["category"] = df["category_binary"].replace({0: 'cat', 1: 'dog'}) 
train_df = df

# Test Dataframe
test_filenames = os.listdir("Datasets/test1/")
test_df = pd.DataFrame({
    'filename': test_filenames
})

In [ ]:
# Part 1 - Data Preprocessing

# Preprocessing the Training set
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

training_set  = train_datagen.flow_from_dataframe(
    train_df, 
    "Datasets/train/", 
    x_col='filename',
    y_col='category',
    target_size=(224, 224),
    class_mode='binary',
    batch_size=32
)

# Preprocessing the Test set
test_datagen  = ImageDataGenerator(rescale=1./255)
test_set  = test_gen.flow_from_dataframe(
    test_df, 
    "Datasets/test1/", 
    x_col='filename',
    y_col=None,
    class_mode=None,
    target_size = (224, 224), 
    batch_size = 32, 
    shuffle=False
)

In [ ]:
# Part 2 - Building the CNN
# Initialising the CNN
cnn = tf.keras.models.Sequential()

# Step 1 - Convolution
cnn.add(tf.keras.layers.Conv2D(filters=32,padding="same",kernel_size=3, activation='relu', strides=2, input_shape=[224, 224, 3]))

# Step 2 - Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding a second convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=32,padding='same',kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Step 3 - Flattening
cnn.add(tf.keras.layers.Flatten())

# Step 4 - Full Connection
cnn.add(tf.keras.layers.Dense(units=448, activation='relu'))

# Step 5 - Output Layer
#cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
## For Binary Classification
cnn.add(Dense(1, kernel_regularizer=tf.keras.regularizers.l2(0.01),activation
             ='linear'))

cnn.summary()

In [ ]:
# Part 3 - Training the CNN

# Compiling the CNN
cnn.compile(optimizer = 'adam', loss = 'hinge', metrics = ['accuracy'])

# Training the CNN on the Training set and evaluating it on the Test set
r=cnn.fit(x = training_set, validation_data = test_set, epochs = 10)

In [ ]:
model_json = cnn.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
cnn.save_weights("model.h5")
print("Saved model to disk")

In [ ]:
# load json and create model
import json
from keras.models import model_from_json

with open('model.json') as f:
    json_file = json.load(f)
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")

# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
loaded_model.summary()

In [ ]:
from tensorflow.keras.models import load_model

cnn.save('model_cat_dog.h5')

In [ ]:
from tensorflow.keras.models import load_model
 
# load model
model = load_model('model_cat_dog.h5')

In [ ]:
model.summary()

In [ ]:
directory = "datasets/train/dog.10.jpg"
sample_image = load_img(directory)
plt.imshow(sample_image)

test_image = image_process.load_img(directory)
test_image = test_image.resize((224, 224))
test_image = image_process.img_to_array(test_image)
test_image=test_image/255
test_image = np.expand_dims(test_image, axis = 0)
result = loaded_model.predict(test_image)

result
if result[0]<0:
    print("The image classified is cat")
else:
    print("The image classified is dog")

In [ ]:
result

In [ ]:
result